In [113]:
from police_api import PoliceAPI
import pandas as pd
import re
import numpy as np

In [2]:
api = PoliceAPI()

In [3]:
all_forces = api.get_forces()     #assigning all england forces to variable

In [4]:
london_force = all_forces[24] #assigning london FORCES in variable

In [5]:
london_nei = london_force.neighbourhoods     #assigning neighbourhoods in london in variablr

In [7]:
df_code = pd.DataFrame({'code': london_nei})       #data frame with NH code
df_code.head()

,code
0,<Neighbourhood> E05000026
1,<Neighbourhood> E05000455
2,<Neighbourhood> E05000630
3,<Neighbourhood> E05000214
4,<Neighbourhood> E05009388


In [8]:
df_code['Neighbourhood'] = df_code.apply(lambda x: x['code'].name, axis=1)

#importing neighbourhood names in the dataframe

df_code.head()

,code,Neighbourhood
0,<Neighbourhood> E05000026,Abbey
1,<Neighbourhood> E05000455,Abbey
2,<Neighbourhood> E05000630,Abbey Road
3,<Neighbourhood> E05000214,Abbey Wood
4,<Neighbourhood> E05009388,Abingdon


In [ ]:
#df_code.drop(columns=[0],inplace=True) 
#df_code.set_index('code',inplace=True)
#df['Neighbourhood'].crimes[1]

In [9]:
df_code.code=df_code.code.astype(str)  #cleaning NH column
df_code['code']=df_code['code'].apply(lambda x:x.split(">")[1])
df_code

,code,Neighbourhood
0,E05000026,Abbey
1,E05000455,Abbey
2,E05000630,Abbey Road
3,E05000214,Abbey Wood
4,E05009388,Abingdon
...,...,...
653,E05000230,Woolwich Riverside
654,E05000572,Worcester Park
655,E05000265,Wormholt and White City
656,E05000344,Yeading


In [10]:
new_crimes=[]
neighb=[]
officers=[]
neii=[]
longitudes=[]
latitudes=[]
for i in london_nei:
    for z in i.officers:
        neii.append(i.name)
        officers.append(z)   
    for j in i.crimes:       
        neighb.append(i.name)
        new_crimes.append(j.category)
        longitudes.append(j.location.longitude)
        latitudes.append(j.location.latitude)

In [125]:
df_crimes=pd.DataFrame(neighb)  # New Data frame with(crimes,location)
df_crimes['crimes']=new_crimes
df_crimes['latitude']=latitudes
df_crimes['longitude']=longitudes
df_crimes

,0,crimes,latitude,longitude
0,Abbey,<CrimeCategory> Anti-social behaviour,51.538296,0.087049
1,Abbey,<CrimeCategory> Anti-social behaviour,51.538296,0.087049
2,Abbey,<CrimeCategory> Anti-social behaviour,51.542615,0.081852
3,Abbey,<CrimeCategory> Anti-social behaviour,51.540150,0.082375
4,Abbey,<CrimeCategory> Anti-social behaviour,51.541146,0.082002
...,...,...,...,...
85864,Yiewsley,<CrimeCategory> Violence and sexual offences,51.511575,-0.465497
85865,Yiewsley,<CrimeCategory> Violence and sexual offences,51.520429,-0.463844
85866,Yiewsley,<CrimeCategory> Violence and sexual offences,51.514661,-0.466993
85867,Yiewsley,<CrimeCategory> Other crime,51.509944,-0.474198


In [126]:
df_officers=pd.DataFrame(neii)  # New Data frame with(NH , officers)
df_officers['officers']=officers
df_officers.officers=df_officers.officers.astype(str)  #cleaning in officers dataframe
df_officers['officers']=df_officers['officers'].apply(lambda x:x.split(">")[1])
df_officers.rename(columns={0:'Neighbourhood'},inplace=True)

In [127]:
df_crimes.crimes=df_crimes.crimes.astype(str)  #cleaning in crimes dataframe
df_crimes['crimes']=df_crimes['crimes'].apply(lambda x:x.split(">")[1])

In [145]:
df_crimes
df_antianti=df_crimes.loc[df_crimes['crimes']!=' Anti-social behaviour']
df_antianti
df_antianti.to_csv('/Users/islaam/Desktop/anti.csv')

In [129]:
df_officers.rename(columns={0:'Neighbourhood'},inplace=True) 

In [130]:
df_crimes.rename(columns={0:'Neighbourhood'},inplace=True)

In [131]:
crimetypepernh = df_code.merge(df_crimes, how='right').groupby(['Neighbourhood','crimes']).count()
#dataframe for crimes per NH
crimetypepernh.drop(columns=['latitude','longitude'],inplace=True) 
crimetypepernh.rename(columns={'code':'total crimes'},inplace=True)
crimetypepernh

total crimes
Neighbourhood crimes                                     
Abbey          Anti-social behaviour                  154
               Bicycle theft                            8
               Burglary                                28
               Criminal damage and arson               36
               Drugs                                   40
...                                                   ...
Yiewsley       Robbery                                  5
               Shoplifting                             12
               Theft from the person                    2
               Vehicle crime                           16
               Violence and sexual offences            33

[7626 rows x 1 columns]

In [132]:
crimepernh=df_crimes.groupby(['Neighbourhood']).count()
crimepernh.drop(columns=['latitude','longitude'],inplace=True) 
crimepernh = crimepernh.sort_values('crimes',ascending=False)
crimepernh.reset_index(inplace=True)

In [133]:
mix = crimepernh.merge(officerspernh, how='outer').groupby(['Neighbourhood']).sum()
mix


,crimes,officers
Neighbourhood,,
Abbey,351.0,15
Abbey Road,72.0,5
Abbey Wood,185.0,4
Abingdon,108.0,5
Acton Central,141.0,5
...,...,...
Woolwich Riverside,342.0,6
Worcester Park,54.0,4
Wormholt and White City,102.0,3


In [134]:
officerspernh = df_officers.groupby(['Neighbourhood']).count()
#data frame officers per NH
officerspernh = officerspernh.sort_values('officers',ascending=False)
officerspernh.reset_index(inplace=True)

In [137]:
crimesperlondon=df_crimes.groupby(['crimes']).count()
crimesperlondon.drop(columns=['latitude','longitude'],inplace=True) 
crimesperlondon.rename(columns={'Neighbourhood':'total crimes'},inplace=True)
crimesperlondon.sort_values('total crimes',ascending=False,inplace=True)
crimesperlondon


,total crimes
crimes,
Anti-social behaviour,19866
Violence and sexual offences,18499
Vehicle crime,8974
Other theft,8166
Burglary,6198
Public order,4047
Criminal damage and arson,3975
Drugs,3646
Shoplifting,3314
